<a href="https://colab.research.google.com/github/samohtwal/BPR/blob/20221026/BPR_20221026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import torch
import copy
from google.colab import files

from scipy.sparse import csr_matrix, dok_matrix

In [13]:
# load data
raw_data_url = 'https://raw.githubusercontent.com/samohtwal/BPR/main/data/vali-20221026.csv'
user_column_name = 'user_id'
item_column_name = 'item_id'
column_names = [user_column_name, item_column_name]
raw_data = pd.read_csv(raw_data_url, names = column_names, header=0)

print('raw_data dimension:', raw_data.shape)
print('raw_data head: \n', raw_data.head())
print('raw_data.iloc[0][user_column_name]:', raw_data.iloc[0][user_column_name])
print('raw_data.iloc[0][item_column_name]:', raw_data.iloc[0][item_column_name])
print('raw_data.iloc[25054][user_column_name]:', raw_data.iloc[25054][user_column_name])
print('raw_data.iloc[25054][item_column_name]:', raw_data.iloc[25054][item_column_name])

raw_data dimension: (25055, 2)
raw_data head: 
    user_id  item_id
0        6        0
1        7        0
2        9        0
3       15        1
4       28        2
raw_data.iloc[0][user_column_name]: 6
raw_data.iloc[0][item_column_name]: 0
raw_data.iloc[25054][user_column_name]: 2148
raw_data.iloc[25054][item_column_name]: 3057


In [8]:
def create_user_item_rating_matrix(raw_data, user_column_name, item_column_name):

    user_ids = sorted(raw_data[user_column_name].unique())
    n_user = len(user_ids)

    item_ids = sorted(raw_data[item_column_name].unique())
    n_item = len(item_ids)

    user_item_rating_matrix = np.zeros((n_user, n_item), dtype = np.int32)

    '''
    for x in range(result_data.shape[0]-1):
        user_item_rating_matrix[user_ids.index(result_data.iloc[x][user_column_name]), item_ids.index(result_data.iloc[x][item_column_name])] = 1
    '''

    return user_item_rating_matrix

In [9]:
user_item_rating_matrix = create_user_item_rating_matrix(raw_data, user_column_name, item_column_name)
print('user_item_rating_matrix:\n', user_item_rating_matrix)

user_item_rating_matrix:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
